In [51]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
#Twitter credentials for the app
consumer_key='XLlWiRQ8NZuqd8XRLv2nI8TxL'
consumer_secret='RcylzrmlHqSLJ3JqB9UmP3UJ9eTJnIgomIJRssUWKynG9SSTJO'
access_key='124194567-9Ka3HipzL4UuuV8Xq5JC1mVbbKrRi0IcbtvB6UZK'
access_secret='xI2xkSDovyWKodwIo05ptBO0lsd4PzLEtOv6y7VfLVtXI'
 
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

api = tweepy.API(auth)
 
 
#columns of the csv file
COLS = ['Tweet','Subjectivity','Polarity','Sentiment']
df = pd.DataFrame(columns=COLS) 

 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
 
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)
 
 #mrhod clean_tweets()
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
 
    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the beginning of the tweet
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
 
 
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
 
    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
 
    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)
 


In [52]:
# Define the search term and the date_since date as variables
search_words = "#cannabis, #review"
date_since = "2018-11-16"

# Collect tweets
tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(500)

# Iterate and print tweets
for tweet in tweets:  
    original_tweet=tweet.text
    #call clean_tweet method for extra preprocessing
    filtered_tweet=clean_tweets(tweet.text) 
    #pass textBlob method for sentiment calculations
    blob = TextBlob(filtered_tweet)
    Sentiment = blob.sentiment 
    #seperate polarity and subjectivity in to two variables
    polarity = Sentiment.polarity
    subjectivity = Sentiment.subjectivity    
    #set sentiment 
    if polarity > 0: 
        sent='positive'
    elif polarity == 0: 
        sent='neutral'
    else: 
        sent='negative'
   
    df = df.append({'Tweet': '%s' %original_tweet,'Subjectivity': '%s'%subjectivity, 'Polarity': '%s' %polarity,'Sentiment':'%s'%sent}, ignore_index=True)
    
    
df.head()

,Tweet,Subjectivity,Polarity,Sentiment
0,#mdpibrainsci The Effects of #Cannabinoids on ...,0.0,0.0,neutral
1,#steem #blockchain #dapps #frontends and #comm...,0.0,0.0,neutral
2,DM for free stickers!!!\n#Free #Stickers #Weed...,0.8,0.4,positive
3,Thanks for your review Jerry!! \n\n#Insa #MyIn...,0.2,0.2,positive
4,CBD Review: CBDistillery – CBD Pet Tincture 15...,0.0,0.0,neutral


In [53]:
df.to_csv('sentiment3.csv', encoding='utf-8', index=False)